In [ ]:
## With the model we build in the E1 we are going predict the result for this set of data.now things to note are that when training an object for the first time you use fit_transform but after that you only use transform.

import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [95]:
model = load_model('model.h5')
with open('ohe.pkl','rb') as file:
    ohe = pickle.load(file)

with open('leg.pkl','rb') as file:
    leg = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [96]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [97]:
data = pd.DataFrame([input_data])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [98]:
data['Gender'] = leg.transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [100]:
values = ohe.transform(data[['Geography']]).toarray()
values_df = pd.DataFrame(values,columns=ohe.get_feature_names_out(['Geography']))



In [101]:
values_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [103]:
data = data.drop(['Geography'],axis =1)

In [104]:
data = pd.concat([data,values_df],axis =1)

In [105]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [106]:
ds = scaler.transform(data)

In [107]:
ds

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [108]:
prediction = model.predict(ds)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


array([[0.09698071]], dtype=float32)

In [113]:
prediction_proba = prediction[0][0]

In [114]:
prediction_probability

np.float32(0.09698071)

In [115]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
